In [1]:
import os
import sys
import numpy as np
import pandas as pd
import json

sys.path.append('../')
sys.path.append('../src/')
sys.path.append('../spell/')

import Reader
import ParamsExtractor3
import DataPreprocessor
import EncodeCommand
import DeepLearningAnomalyDetection2

2024-09-05 09:19:06.823478: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-05 09:19:06.873046: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-05 09:19:07.665354: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# First type of log file:
# laurel_anomalous_new
fp = '../data/silent/audit.log'
fps = [fp]
fps += [fp + f'.{str(i)}' for i in range(1,11)]
print(fps)

['../data/silent/audit.log', '../data/silent/audit.log.1', '../data/silent/audit.log.2', '../data/silent/audit.log.3', '../data/silent/audit.log.4', '../data/silent/audit.log.5', '../data/silent/audit.log.6', '../data/silent/audit.log.7', '../data/silent/audit.log.8', '../data/silent/audit.log.9', '../data/silent/audit.log.10']


In [3]:
fp = fps[0]

data_list = []
lines_limit = 1000 # limit reading lines, done for memory constraints
i = 0

with open(fp, 'r') as file:
    for line in file:
        i+=1
        if i > lines_limit:
            break
            # Find the index of the first '{'
        index = line.find('{')
        if index != -1:
            json_data = line[index:]
            data = json.loads(json_data)
            data_list.append(data)
                
df = pd.DataFrame(data_list)

In [4]:
df.head()

,ID,SYSCALL,EXECVE,CWD,PATH,PROCTITLE,BPF,SERVICE_STOP,BPRM_FCAPS,SERVICE_START,USER_AUTH,USER_ACCT,CRED_REFR
0,1725526440.986:19054,"{'arch': '0xc000003e', 'syscall': 59, 'success...","{'argc': 3, 'ARGV': ['/usr/bin/sh', '/usr/libe...",{'cwd': '/home/matteo'},"[{'item': 0, 'name': '/usr/libexec/grepconf.sh...","{'ARGV': ['/usr/bin/sh', '/usr/libexec/grepcon...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1725526440.993:19055,"{'arch': '0xc000003e', 'syscall': 59, 'success...","{'argc': 4, 'ARGV': ['grep', '-qsi', '^COLOR.*...",{'cwd': '/home/matteo'},"[{'item': 0, 'name': '/bin/grep', 'inode': 342...","{'ARGV': ['grep', '-qsi', '^COLOR.*none', '/et...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1725526441.000:19056,"{'arch': '0xc000003e', 'syscall': 59, 'success...","{'argc': 3, 'ARGV': ['/usr/bin/sh', '/usr/libe...",{'cwd': '/home/matteo'},"[{'item': 0, 'name': '/usr/libexec/grepconf.sh...","{'ARGV': ['/usr/bin/sh', '/usr/libexec/grepcon...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1725526441.002:19057,"{'arch': '0xc000003e', 'syscall': 59, 'success...","{'argc': 4, 'ARGV': ['grep', '-qsi', '^COLOR.*...",{'cwd': '/home/matteo'},"[{'item': 0, 'name': '/bin/grep', 'inode': 342...","{'ARGV': ['grep', '-qsi', '^COLOR.*none', '/et...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1725526441.009:19058,"{'arch': '0xc000003e', 'syscall': 59, 'success...","{'argc': 3, 'ARGV': ['cat', '/dev/null', '/etc...",{'cwd': '/home/matteo'},"[{'item': 0, 'name': '/bin/cat', 'inode': 3449...","{'ARGV': ['cat', '/dev/null', '/etc/debuginfod...",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
def normalize(df, col):
    if col not in df.columns:
        print(f"Error, column {col} not in df")
        return None
    df_expanded = pd.json_normalize(df[col])
    #print(df_expanded.head())
    df = df.drop(col, axis=1)
    df = df.join(df_expanded, rsuffix=f'_{col}')
    return df

In [6]:
df.columns

Index(['ID', 'SYSCALL', 'EXECVE', 'CWD', 'PATH', 'PROCTITLE', 'BPF',
       'SERVICE_STOP', 'BPRM_FCAPS', 'SERVICE_START', 'USER_AUTH', 'USER_ACCT',
       'CRED_REFR'],
      dtype='object')

In [7]:
df = normalize(df, 'CWD')
df = normalize(df, 'PATH')
df = normalize(df, 'SYSCALL')
df = normalize(df, 'PROCTITLE')
df = normalize(df, 'EXECVE')
df = normalize(df, 0)
df = normalize(df, 1)
print(df.head())

try:
    df['ARGV_PROCTITLE_str'] = df['ARGV_PROCTITLE'].apply(lambda x: ' '.join(x))
except:
    pass

df['unix_time'] = pd.to_numeric(df['ID'].str.split(':').str[0])
df['timedelta'] = df['unix_time'].diff()
df['time'] = pd.to_datetime(df['unix_time'], unit='s')

df['ppid_time'] = pd.to_datetime(pd.to_numeric(df['PPID.EVENT_ID'].str.split(':').str[0]), unit='s')
df['ppid_timedelta'] = df['ppid_time'].diff()

df['pid_time'] = pd.to_datetime(pd.to_numeric(df['PID.EVENT_ID'].str.split(':').str[0]), unit='s')
df['pid_timedelta'] = df['pid_time'].diff()

df = df.drop(['ID', 'PID.EVENT_ID', 'PPID.EVENT_ID', 'unix_time', 'ARGV_PROCTITLE', 'time', 'pid_time', 'ppid_time'], axis=1)

                     ID  BPF SERVICE_STOP BPRM_FCAPS SERVICE_START USER_AUTH  \
0  1725526440.986:19054  NaN          NaN        NaN           NaN       NaN   
1  1725526440.993:19055  NaN          NaN        NaN           NaN       NaN   
2  1725526441.000:19056  NaN          NaN        NaN           NaN       NaN   
3  1725526441.002:19057  NaN          NaN        NaN           NaN       NaN   
4  1725526441.009:19058  NaN          NaN        NaN           NaN       NaN   

  USER_ACCT CRED_REFR           cwd  \
0       NaN       NaN  /home/matteo   
1       NaN       NaN  /home/matteo   
2       NaN       NaN  /home/matteo   
3       NaN       NaN  /home/matteo   
4       NaN       NaN  /home/matteo   

                                                   2  ... rdev_1  \
0  {'item': 2, 'name': '/lib64/ld-linux-x86-64.so...  ...  00:00   
1                                               None  ...  00:00   
2  {'item': 2, 'name': '/lib64/ld-linux-x86-64.so...  ...  00:00   
3           

In [8]:
for col in df.columns:
    print(f'{col} : {df.iloc[1][col]}')

BPF : nan
SERVICE_STOP : nan
BPRM_FCAPS : nan
SERVICE_START : nan
USER_AUTH : nan
USER_ACCT : nan
CRED_REFR : nan
cwd : /home/matteo
2 : None
arch : 0xc000003e
syscall : 59.0
success : yes
exit : 0.0
items : 2.0
ppid : 30536.0
pid : 30537.0
auid : 1000.0
uid : 0.0
gid : 0.0
euid : 0.0
suid : 0.0
fsuid : 0.0
egid : 0.0
sgid : 0.0
fsgid : 0.0
tty : pts0
ses : 3.0
comm : grep
exe : /usr/bin/grep
subj : unconfined_u:unconfined_r:unconfined_t:s0-s0:c0.c1023
key : exec_log
ARCH : x86_64
SYSCALL : execve
AUID : matteo
UID : root
GID : root
EUID : root
SUID : root
FSUID : root
EGID : root
SGID : root
FSGID : root
ARGV : ['0x5626436311d0', '0x56264362dee0', '0x56264362d940', '0x8']
UID_GROUPS : ['root']
SCRIPT : nan
PPID.comm : grepconf.sh
PPID.exe : /usr/bin/bash
PPID.ppid : 30529.0
PPID.START_TIME : nan
argc : 4.0
ARGV_EXECVE : ['grep', '-qsi', '^COLOR.*none', '/etc/GREP_COLORS']
ENV.LD_LIBRARY_PATH : nan
ENV.LD_PRELOAD : nan
item : 0.0
name : /bin/grep
inode : 34258834.0
dev : fd:00
mode : 0

In [12]:
for i in range(0,20):
    print(f'UID_GROUPS : {df.iloc[i]["UID_GROUPS"]}')
print(type(df.iloc[1]["UID_GROUPS"]))

UID_GROUPS : ['root']
UID_GROUPS : ['root']
UID_GROUPS : ['root']
UID_GROUPS : ['root']
UID_GROUPS : ['root']
UID_GROUPS : ['root']
UID_GROUPS : ['root']
UID_GROUPS : ['root']
UID_GROUPS : ['root']
UID_GROUPS : ['root']
UID_GROUPS : ['root']
UID_GROUPS : ['root']
UID_GROUPS : ['root']
UID_GROUPS : ['root']
UID_GROUPS : nan
UID_GROUPS : ['root']
UID_GROUPS : ['root']
UID_GROUPS : nan
UID_GROUPS : ['root']
UID_GROUPS : nan
<class 'list'>


In [10]:
columns = ['auid', 'uid', 'gid', 'euid', 'suid', 'fsuid', 'egid', 'sgid', 'fsgid']

def check_row_id_num(row):
    return pd.Series({col: row[col] == 0 for col in columns})

checks = df.apply(check_row_id_num, axis=1)
df['num_id_anomalies'] = checks.sum(axis=1)
df = df.drop(columns, axis=1)

In [13]:
columns = ['AUID', 'UID', 'GID', 'EUID', 'SUID', 'FSUID', 'EGID', 'SGID', 'FSGID']

# Use apply to efficiently check values for each row
def check_id_row(row):
    try:
        x = pd.Series({col: int(not(row[col] in row['UID_GROUPS'])) for col in columns})
    except:
        x = None
    return x

checks = df.apply(check_id_row, axis=1)

df['id_anomalies'] = checks.sum(axis=1)
df = df.drop(columns, axis=1)
df = df.drop('UID_GROUPS', axis=1)

In [14]:
df.head()

,BPF,SERVICE_STOP,BPRM_FCAPS,SERVICE_START,USER_AUTH,USER_ACCT,CRED_REFR,cwd,2,arch,...,cap_fe_1,cap_fver_1,cap_frootid_1,OUID_1,OGID_1,timedelta,ppid_timedelta,pid_timedelta,num_id_anomalies,id_anomalies
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/home/matteo,"{'item': 2, 'name': '/lib64/ld-linux-x86-64.so...",0xc000003e,...,0.0,0x0,0,root,root,NaN,NaT,NaT,8,1.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/home/matteo,None,0xc000003e,...,0.0,0x0,0,root,root,0.007,0 days 00:00:00.121999979,0 days 00:00:00.006999970,8,1.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/home/matteo,"{'item': 2, 'name': '/lib64/ld-linux-x86-64.so...",0xc000003e,...,0.0,0x0,0,root,root,0.007,-1 days +23:59:59.878000021,0 days 00:00:00.006999969,8,1.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/home/matteo,None,0xc000003e,...,0.0,0x0,0,root,root,0.002,0 days 00:00:00.135999918,0 days 00:00:00.002000093,8,1.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/home/matteo,None,0xc000003e,...,0.0,0x0,0,root,root,0.007,NaT,0 days 00:00:00.006999970,8,1.0


In [15]:
for col in df.columns:
    print(f'{col} : {df.iloc[1][col]}')

BPF : nan
SERVICE_STOP : nan
BPRM_FCAPS : nan
SERVICE_START : nan
USER_AUTH : nan
USER_ACCT : nan
CRED_REFR : nan
cwd : /home/matteo
2 : None
arch : 0xc000003e
syscall : 59.0
success : yes
exit : 0.0
items : 2.0
ppid : 30536.0
pid : 30537.0
tty : pts0
ses : 3.0
comm : grep
exe : /usr/bin/grep
subj : unconfined_u:unconfined_r:unconfined_t:s0-s0:c0.c1023
key : exec_log
ARCH : x86_64
SYSCALL : execve
ARGV : ['0x5626436311d0', '0x56264362dee0', '0x56264362d940', '0x8']
SCRIPT : nan
PPID.comm : grepconf.sh
PPID.exe : /usr/bin/bash
PPID.ppid : 30529.0
PPID.START_TIME : nan
argc : 4.0
ARGV_EXECVE : ['grep', '-qsi', '^COLOR.*none', '/etc/GREP_COLORS']
ENV.LD_LIBRARY_PATH : nan
ENV.LD_PRELOAD : nan
item : 0.0
name : /bin/grep
inode : 34258834.0
dev : fd:00
mode : 0o100755
ouid : 0.0
ogid : 0.0
rdev : 00:00
obj : system_u:object_r:bin_t:s0
nametype : NORMAL
cap_fp : 0x0
cap_fi : 0x0
cap_fe : 0.0
cap_fver : 0x0
cap_frootid : 0
OUID : root
OGID : root
item_1 : 1.0
name_1 : /lib64/ld-linux-x86-64.s

In [18]:
# filter final columns
columns_to_keep = ['cwd', 'exit', 'items', 'ppid', 'pid', 'comm', 'timedelta', 'pid_timedelta', 'ppid_timedelta', 'id_anomalies', 'num_id_anomalies']
df = df [columns_to_keep]

In [19]:
df.iloc[1]

cwd                              /home/matteo
exit                                      0.0
items                                     2.0
ppid                                  30536.0
pid                                   30537.0
comm                                     grep
timedelta                               0.007
pid_timedelta       0 days 00:00:00.006999970
ppid_timedelta      0 days 00:00:00.121999979
id_anomalies                              1.0
num_id_anomalies                            8
Name: 1, dtype: object

In [20]:
os.getcwd()

'/home/rocky/repo/Thesis/notebooks_dataset_creation'

In [23]:
# Save this df to file
save_fp = '../data/silent/save1.csv'
df.to_csv(save_fp, index=False)

In [52]:
# Final Preprocessing Function
def preprocess_laurel(df):
    
    df = normalize(df, 'CWD')
    df = normalize(df, 'PATH')
    df = normalize(df, 'SYSCALL')
    df = normalize(df, 'PROCTITLE')
    df = normalize(df, 'EXECVE')
    df = normalize(df, 0)
    df = normalize(df, 1)
    print(df.head())

    columns_to_keep = ['cwd', 'exit', 'items', 'ppid', 'pid', 'comm', 'timedelta', 'pid_timedelta', 'ppid_timedelta', 'id_anomalies', 'num_id_anomalies']
    columns_to_drop = ['ID', 'PID.EVENT_ID', 'PPID.EVENT_ID', 'unix_time', 'ARGV_PROCTITLE', 'time', 'pid_time', 'ppid_time']
    
    try:
        df['ARGV_PROCTITLE_str'] = df['ARGV_PROCTITLE'].apply(lambda x: ' '.join(x))
    except:
        pass
    
    df['unix_time'] = pd.to_numeric(df['ID'].str.split(':').str[0])
    df['timedelta'] = df['unix_time'].diff()
    df['time'] = pd.to_datetime(df['unix_time'], unit='s')

    try:
        df['ppid_time'] = pd.to_datetime(pd.to_numeric(df['PPID.EVENT_ID'].str.split(':').str[0]), unit='s')
        df['ppid_timedelta'] = df['ppid_time'].diff()
        columns_to_keep.remove('ppid_timedelta')
        columns_to_drop.remove('ppid_time')
    except:
        columns_to_drop.remove('ppid_time')
        columns_to_keep.remove('ppid_timedelta')
        columns_to_drop.remove('PPID.EVENT_ID')

    try:
        df['pid_time'] = pd.to_datetime(pd.to_numeric(df['PID.EVENT_ID'].str.split(':').str[0]), unit='s')
        df['pid_timedelta'] = df['pid_time'].diff()
        columns_to_keep.remove('pid_timedelta')
        columns_to_drop.remove('pid_time')
    except:
        columns_to_drop.remove('pid_time')
        columns_to_keep.remove('pid_timedelta')
        columns_to_drop.remove('PID.EVENT_ID')
    
    df = df.drop(columns_to_drop, axis=1)

    columns = ['auid', 'uid', 'gid', 'euid', 'suid', 'fsuid', 'egid', 'sgid', 'fsgid']
    
    def check_row_id_num(row):
        return pd.Series({col: row[col] == 0 for col in columns})
    
    checks = df.apply(check_row_id_num, axis=1)
    df['num_id_anomalies'] = checks.sum(axis=1)
    df = df.drop(columns, axis=1)

    columns = ['AUID', 'UID', 'GID', 'EUID', 'SUID', 'FSUID', 'EGID', 'SGID', 'FSGID']
    
    # Use apply to efficiently check values for each row
    def check_id_row(row):
        try:
            x = pd.Series({col: int(not(row[col] in row['UID_GROUPS'])) for col in columns})
        except:
            x = None
        return x
    
    checks = df.apply(check_id_row, axis=1)
    
    df['id_anomalies'] = checks.sum(axis=1)
    df = df.drop(columns, axis=1)
    df = df.drop('UID_GROUPS', axis=1)

    # filter final columns
    df = df [columns_to_keep]

    return df

In [53]:
def read(fp):
    data_list = []
    lines_limit = 1000 # limit reading lines, done for memory constraints
    i = 0
    
    with open(fp, 'r') as file:
        for line in file:
            i+=1
            if i > lines_limit:
                break
                # Find the index of the first '{'
            index = line.find('{')
            if index != -1:
                json_data = line[index:]
                data = json.loads(json_data)
                data_list.append(data)
                    
    df = pd.DataFrame(data_list)
    return df

In [54]:
def save(df, save_fp):
    # Save this df to file
    #save_fp = '../data/laurel_anomalous_new/save1.csv'
    df.to_csv(save_fp, index=False)

In [55]:
for fp in fps:
    df = read(fp)
    df = preprocess_laurel(df)
    save(df, fp + '.csv')

                     ID  BPF SERVICE_STOP BPRM_FCAPS SERVICE_START USER_AUTH  \
0  1725526440.986:19054  NaN          NaN        NaN           NaN       NaN   
1  1725526440.993:19055  NaN          NaN        NaN           NaN       NaN   
2  1725526441.000:19056  NaN          NaN        NaN           NaN       NaN   
3  1725526441.002:19057  NaN          NaN        NaN           NaN       NaN   
4  1725526441.009:19058  NaN          NaN        NaN           NaN       NaN   

  USER_ACCT CRED_REFR           cwd  \
0       NaN       NaN  /home/matteo   
1       NaN       NaN  /home/matteo   
2       NaN       NaN  /home/matteo   
3       NaN       NaN  /home/matteo   
4       NaN       NaN  /home/matteo   

                                                   2  ... rdev_1  \
0  {'item': 2, 'name': '/lib64/ld-linux-x86-64.so...  ...  00:00   
1                                               None  ...  00:00   
2  {'item': 2, 'name': '/lib64/ld-linux-x86-64.so...  ...  00:00   
3           

In [ ]:
# merge the csv files